Здравствуйте.

# Задание
Ниже обучается и оцениваться модель. 

Нужно перевести этот в Pipeline (вам понадобится VectorAssembler), а затем оценить MAE с помощью spark. И попробовать потюнить параметры модели.


In [1]:
!pip install pyspark

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 281.3 MB 64 kB/s 
     |████████████████████████████████| 199 kB 78.3 MB/s 
  Created wheel for pyspark: filename=pyspark-3.3.0-py2.py3-none-any.whl size=281764026 sha256=5ceec946c6610e45781a02b0b2cb3bbd3cf93ca74b85e56287d49232906468f0
  Stored in directory: /root/.cache/pip/wheels/7a/8e/1b/f73a52650d2e5f337708d9f6a1750d451a7349a867f928b885
Successfully built pyspark


In [2]:
# https://scikit-learn.org/stable/datasets/toy_dataset.html#boston-dataset

from pyspark.ml.evaluation import RegressionEvaluator, BinaryClassificationEvaluator

In [3]:
import pandas as pd
from sklearn.datasets import load_diabetes, load_iris, load_boston
from sklearn.metrics import mean_absolute_error

from pyspark.sql import SparkSession
from pyspark.ml.linalg import Vectors, VectorUDT
from pyspark.sql import functions as F
from pyspark.sql.types import *
from pyspark.ml.regression import RandomForestRegressor

spark = SparkSession.builder\
    .master("local[2]")\
    .appName("Lesson_2")\
    .config("spark.executor.instances",2)\
    .config("spark.executor.memory",'2g')\
    .config("spark.executor.cores",1)\
    .getOrCreate()
sc = spark.sparkContext

In [4]:
data = load_boston()
dataset = pd.DataFrame(data['data'], columns=data['feature_names'])
dataset['target'] = data['target']

cols_to_vector = F.udf(lambda l: Vectors.dense(l), VectorUDT())

spark_dataset = spark.createDataFrame(dataset).select(cols_to_vector(F.array(*data['feature_names'])).alias('features'), 'target').cache()

/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function load_boston is deprecated; `load_boston` is deprecated in 1.0 and will be removed in 1.2.

    The Boston housing prices dataset has an ethical problem. You can refer to
    the documentation of this function for further details.

    The scikit-learn maintainers therefore strongly discourage the use of this
    dataset unless the purpose of the code is to study and educate about
    ethical issues in data science and machine learning.

    In this special case, you can fetch the dataset from the original
    source::

        import pandas as pd
        import numpy as np


        data_url = "http://lib.stat.cmu.edu/datasets/boston"
        raw_df = pd.read_csv(data_url, sep="\s+", skiprows=22, header=None)
        data = np.hstack([raw_df.values[::2, :], raw_df.values[1::2, :2]])
        target = raw_df.values[1::2, 2]

    Alternative datasets include the California housing dataset (i.e.

In [17]:
spark_dataset.show(10)

+--------------------+------+
|            features|target|
+--------------------+------+
|[0.00632,18.0,2.3...|  24.0|
|[0.02731,0.0,7.07...|  21.6|
|[0.02729,0.0,7.07...|  34.7|
|[0.03237,0.0,2.18...|  33.4|
|[0.06905,0.0,2.18...|  36.2|
|[0.02985,0.0,2.18...|  28.7|
|[0.08829,12.5,7.8...|  22.9|
|[0.14455,12.5,7.8...|  27.1|
|[0.21124,12.5,7.8...|  16.5|
|[0.17004,12.5,7.8...|  18.9|
+--------------------+------+
only showing top 10 rows



In [5]:
train, test = spark_dataset.randomSplit([0.7, 0.3])

lr = RandomForestRegressor(labelCol='target')
lr = lr.fit(train)
train_predictions = lr.transform(train)
test_predictions = lr.transform(test)

In [6]:
# Заменить нужно эту часть
pandas_train_predictions = train_predictions.toPandas()
pandas_test_predictions = test_predictions.toPandas()

print(f'''
    Scores:: 
        train: {mean_absolute_error(
            pandas_train_predictions.target, 
            pandas_train_predictions.prediction)}, 
        test: {mean_absolute_error(
            pandas_test_predictions.target, 
            pandas_test_predictions.prediction)}
    ''')


    Scores:: 
        train: 1.9059847843296451, 
        test: 2.413143131446219
    


Используйте RegressionEvaluator. Evaluation ипортируется следующим образом:


```
from pyspark.ml.evaluation import RegressionEvaluator, BinaryClassificationEvaluator
```

В частности [RegressionEvaluator](https://spark.apache.org/docs/latest/api/python/reference/api/pyspark.ml.evaluation.RegressionEvaluator.html#pyspark.ml.evaluation.RegressionEvaluator.metricName)

# 1 Переведите в пайплайн

In [32]:
from pyspark.ml import Pipeline

rf = RandomForestRegressor(labelCol='target')

stages=[rf]

pipeline = Pipeline(stages=stages)
pipeModel = pipeline.fit(train)
tr = pipeModel.transform(train)
tr.show(10)

+--------------------+------+------------------+
|            features|target|        prediction|
+--------------------+------+------------------+
|[0.00632,18.0,2.3...|  24.0|27.277383577495726|
|[0.01311,90.0,1.2...|  35.4|35.335081378512164|
|[0.0136,75.0,4.0,...|  18.9| 19.95572424394304|
|[0.01432,100.0,1....|  31.6| 30.21658744407688|
|[0.01439,60.0,2.9...|  29.1|28.998004048993494|
|[0.01778,95.0,1.4...|  32.9| 34.34757004713893|
|[0.02009,95.0,2.6...|  50.0| 45.86032329074435|
|[0.02055,85.0,0.7...|  24.7|24.263540374778838|
|[0.02177,82.5,2.0...|  42.3| 43.79668043360148|
|[0.02187,60.0,2.9...|  31.1|29.106833609930213|
+--------------------+------+------------------+
only showing top 10 rows



# 2 Оцените модель

In [33]:
from pyspark.ml.evaluation import RegressionEvaluator
eval_mae = RegressionEvaluator(
    predictionCol='prediction',
    labelCol='target',
    metricName='mae')
print(eval_mae.evaluate(pipeModel.transform(test)))

2.4131431314462204


# 3 Улучшите модель

В этом вам может помочь gridsearch

```
from pyspark.ml.tuning import TrainValidationSplit, ParamGridBuilder


paramgrid = ParamGridBuilder()\. 
.addGrid()\. 
.addGrid().build(). 
TrainValidationSplit(
    estimator = pipe,  
    evaluator = evaluator,   
    estimatorParamMaps = paramgrid). 
```

In [40]:
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.tuning import TrainValidationSplit, ParamGridBuilder

paramGrid = ParamGridBuilder()\
    .addGrid(rf.numTrees, [50, 100, 150])\
    .addGrid(rf.maxDepth, [2, 4, 6, 8])\
    .addGrid(rf.maxBins, [1000, 5000])\
    .build()

tvs = TrainValidationSplit(
    estimator=rf,
    evaluator=eval_mae,
    estimatorParamMaps=paramGrid)

model_tuned = tvs.fit(train).bestModel
print(eval_mae.evaluate(model_tuned.transform(test)))

2.1143455441435783


In [43]:
model_tuned.extractParamMap()

{Param(parent='RandomForestRegressor_3bbc8c1f6481', name='bootstrap', doc='Whether bootstrap samples are used when building trees.'): True,
 Param(parent='RandomForestRegressor_3bbc8c1f6481', name='cacheNodeIds', doc='If false, the algorithm will pass trees to executors to match instances with nodes. If true, the algorithm will cache node IDs for each instance. Caching can speed up training of deeper trees. Users can set how often should the cache be checkpointed or disable it by setting checkpointInterval.'): False,
 Param(parent='RandomForestRegressor_3bbc8c1f6481', name='checkpointInterval', doc='set checkpoint interval (>= 1) or disable checkpoint (-1). E.g. 10 means that the cache will get checkpointed every 10 iterations. Note: this setting will be ignored if the checkpoint directory is not set in the SparkContext.'): 10,
 Param(parent='RandomForestRegressor_3bbc8c1f6481', name='featureSubsetStrategy', doc="The number of features to consider for splits at each tree node. Supporte